In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [17]:
data=pd.read_csv('../raw_data/states_2.csv')
data.head()

,Year,Jurisdiction,Gender,Average scale score
0,2022,National,Male,239
1,2022,National,Female,233
2,2022,Alabama,Male,233
3,2022,Alabama,Female,228
4,2022,Alaska,Male,224


In [18]:
## data clean
data.replace('—', np.nan, inplace=True)
data.replace('‡', np.nan, inplace=True)
data['Year'] = pd.to_numeric(data['Year'], errors='coerce')
data.dropna(subset=['Year'], inplace=True)  # Drop rows where 'Year' is NaN
data['Year'] = data['Year'].astype(int)
data['Average scale score'] = pd.to_numeric(data['Average scale score'], errors='coerce')

In [19]:
# only 2000 years later
data = data[data['Year'] > 2000]
data = data[data['Jurisdiction'] != 'DoDEA']
data = data[data['Jurisdiction'] != 'Puerto Rico']
data = data[data['Jurisdiction'] != 'District of Columbia']

In [20]:
data['Year'] = data['Year'].astype(str)

In [21]:
null_exists = data['Average scale score'].isnull().sum()
null_exists

0

In [22]:
national= data[data['Jurisdiction'] == 'National']
national

,Year,Jurisdiction,Gender,Average scale score
0,2022,National,Male,239.0
1,2022,National,Female,233.0
108,2019,National,Male,242.0
109,2019,National,Female,239.0
216,2017,National,Male,241.0
217,2017,National,Female,239.0
324,2015,National,Male,241.0
325,2015,National,Female,239.0
432,2013,National,Male,242.0
433,2013,National,Female,241.0


In [23]:
test= data[data['Year'] == '2000']
test

,Year,Jurisdiction,Gender,Average scale score


In [24]:
# Calculate the national average for each year and gender
national_averages = data[data['Jurisdiction'] == 'National'].groupby(['Year', 'Gender'])['Average scale score'].mean().reset_index()
# Merge the national averages back into the original dataset on 'Year' and 'Gender'
data = pd.merge(data, national_averages, on=['Year', 'Gender'], how='left', suffixes=('', '_national'))
#data['Average scale score_national']=data['Average scale score_national'].astype(int)

In [25]:
# Fill null values in 'Average scale score' with the corresponding national average
data['Average scale score'].fillna(data['Average scale score_national'], inplace=True)

# Remove the auxiliary column used for filling nulls
data.drop(columns=['Average scale score_national'], inplace=True)

In [26]:
null_exists = data['Average scale score'].isnull().sum()
null_exists

0

In [27]:
#data['Average scale score']=data['Average scale score'].astype(float)

In [28]:
data

,Year,Jurisdiction,Gender,Average scale score
0,2022,National,Male,239.0
1,2022,National,Female,233.0
2,2022,Alabama,Male,233.0
3,2022,Alabama,Female,228.0
4,2022,Alaska,Male,224.0
...,...,...,...,...
1015,2003,West Virginia,Female,230.0
1016,2003,Wisconsin,Male,238.0
1017,2003,Wisconsin,Female,235.0
1018,2003,Wyoming,Male,242.0


In [29]:
data.to_csv('../clean_data/after_clean.csv',index=False)